In [1]:
print("OK all backend is written here")

OK all backend is written here


In [2]:
%pwd

'c:\\Users\\PUJA KUMARI\\Desktop\\medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PUJA KUMARI\\Desktop\\medical-chatbot'

In [6]:
!pip install langchain

Unable to create process using 'C:\Users\PUJA KUMARI\.conda\envs\testingopenai\python.exe "C:\Users\PUJA KUMARI\.conda\envs\testingopenai\Scripts\pip-script.py" install langchain'


In [7]:
#import all the libraries
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


ModuleNotFoundError: No module named 'langchain'

In [11]:
#Extract data from the PDf
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader
                           )
    #call the load clas and store it in documents
    documents=loader.load()

    #return this document
    return documents

In [12]:
extracted_data=load_pdf("data/")

In [14]:
#data loaded as document
 #extracted_data

In [17]:
#create text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [18]:
#call the function and see the length of the chunks
text_chunks=text_split(extracted_data)
print("Length of my chunk:",len(text_chunks))

Length of my chunk: 7020


In [ ]:
#text_chunks

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [21]:
#call the function
embeddings=download_hugging_face_embeddings()

In [22]:
#print the embedding object
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [23]:
# test the embedding
query_result=embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [25]:
#query_result

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
# create the pinecone index
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")

In [ ]:
retrieved_docs

In [ ]:
# here we use one llm for getting the answer
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1246250385.py, line 2)

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

NameError: name 'llm' is not defined

In [ ]:
#ask question 
while True:
    user_input=input(f"Input Prompts:")  #it will take input from the user
    result=qa({"query":user_input})  #ask the question to my llm
    print("Response : ",result["result"])  #llm will give us the response